# Huurhuis webscraper

Dit is een coding project dat automatisch de verschillende websites van makelaars scraped en doormailt naar mij.

## PLAN:

### Database (Postgress):

BrokerAgencies
CREATE TABLE BrokerAgencies (
    MakelaardijID SERIAL PRIMARY KEY,
    Naam TEXT NOT NULL,
    Link TEXT NOT NULL
    BrokerName TEXT NOT NULL,
    Hyperlink TEXT NOT NULL,
    BrokerId SERIAL PRIMARY KEY
);

Property
CREATE TABLE Property (
    MakelaardijID INT REFERENCES Makelaardij(MakelaardijID),
    Adres TEXT NOT NULL,
    Link TEXT NOT NULL,
    ToegevoegdOp DATE NOT NULL,
    NaamDorpStad TEXT NOT NULL,
    Huurprijs TEXT NOT NULL,
    Oppervlakte TEXT NOT NULL,
    BrokerId INT REFERENCES BrokerAgencies(BrokerId),
    Hyperlink TEXT NOT NULL,
    Price TEXT NOT NULL,
    Size TEXT NOT NULL
);

#### CRUD:
Create:
CreateNewBrokerAgency
CreateNewRentalProperty

Read:
GetBrokerAgency
GetPropertiesForBroker
GetAllProperties
GetAllPropertiesForLocation

Update: 
UpdatePropertie
UpdateBrokerAgency

Delete: 
RemoveProperty
RemoveBrokerAgency



### Webscraper (python):

Moet kunnen ophalen:
- Link naar appartement
- Adress 
- Huurprijs
- Oppervlakte



### Communicatielaag (python):

communiceren tussen webscraper en db.
uit locatie naam van dorp halen.
logica voor aanmaken van items in de db.
Controlleren op nieuwe appartementen.
Opstellen mail met nieuwe appartementen en wat basis data.



### Mail service (python):

Moet een mail kunnen versturen naar mijn of een andere mail.



Moet kunnen draaien in een container die een of meerdere malen per dag kan worden afgevuurd (Liefst rond 12 uur smiddags en 6 uur savonds).
kan blijkbaar via crontab op linux


## Imports:

In [3]:
import psycopg2
from psycopg2 import sql
from datetime import date
from dataclasses import dataclass
from typing import List, Optional
import requests
from bs4 import BeautifulSoup
from abc import ABC, abstractmethod
import re
from urllib.parse import urljoin
from typing import Dict, List, Optional, Union, Any
import logging
import threading
from concurrent.futures import ThreadPoolExecutor, as_completed
from queue import Queue
import logging
from datetime import datetime

# Import configuration
from config import DATABASE, EMAIL

# Import mail service
from mail_service import MailService

# Import data access
from data_access import DataAccess, BrokerAgency, Property

# Import logging service
from log_service import LogService, get_logger

## Main:

In [4]:
# Importeer de connector
from connector import Connector


# Uitbreiden van connector met factory functies voor onze datamodellen
class Huurhuisconnector(Connector):
    def _create_broker_agency(self, id, naam, link):
        """Factory functie die een BrokerAgency object aanmaakt met de klasse uit data_access.py."""
        return BrokerAgency(id=id, naam=naam, link=link)

    def _create_property(
        self,
        makelaardij_id,
        adres,
        link,
        toegevoegd_op,
        naam_dorp_stad,
        huurprijs,
        oppervlakte,
    ):
        """Factory functie die een Property object aanmaakt met de klasse uit data_access.py."""
        return Property(
            makelaardij_id=makelaardij_id,
            adres=adres,
            link=link,
            toegevoegd_op=toegevoegd_op,
            naam_dorp_stad=naam_dorp_stad,
            huurprijs=huurprijs,
            oppervlakte=oppervlakte,
        )


def run_scraper_proces():
    """Voer het volledige scraper proces uit."""
    # Initialize logging service
    log_service = LogService()
    logger = get_logger("HuurhuisWebscraper")

    # Log application start
    log_service.log_app_start()

    # Initialiseer de database-verbinding
    db = DataAccess()

    # Initialiseer de connector met onze aangepaste versie
    communicatie = Huurhuisconnector(db)

    # Lijst met te verwerken makelaars en hun scraper-type
    makelaars = [
        {
            "naam": "Van Roomen Van de Bunt NVM Makelaars",
            "type": "vdbunt",
            "url": "https://www.vdbunt.nl/aanbod/woningaanbod/huur/",
        },
        {
            "naam": "Pararius",
            "type": "pararius",
            "url": "https://www.pararius.nl/huurwoningen/leusden/",
        },
        {
            "naam": "Zonnenberg Makelaardij",
            "type": "zonnenberg",
            "url": "https://zonnenbergmakelaardij.nl/woningaanbod/huur/",
        },
        {
            "naam": "Ditters Makelaars",
            "type": "ditters",
            "url": "https://www.ditters.nl/woningaanbod/?filter%5Bcategory%5D=%2FHuur",
        },
        {
            "naam": "InterHouse Utrecht",
            "type": "interhouse-utrecht",
            "url": "https://interhouse.nl/huurwoningen/?location_id=Utrecht_Algemeen&number_of_results=20&sort=date-desc&display=list",
        },
        {
            "naam": "InterHouse Amersfoort",
            "type": "interhouse-amersfoort",
            "url": "https://interhouse.nl/huurwoningen/?location_id=Amersfoort_Algemeen&number_of_results=20&sort=date-desc&display=list",
        },
        {
            "naam": "VastgoedNederland Veenendaal",
            "type": "vastgoednederland",
            "url": "https://aanbod.vastgoednederland.nl/huurwoningen?q=veenendaal&straal=15000",
        },
    ]

    # Set main thread name for better logging
    threading.current_thread().name = "MainThread"
    logger.info("Starting multithreaded scraper process")

    # Process brokers in parallel and gather results
    # Use max_workers based on number of brokers - for 4 brokers, use 4 threads
    max_workers = len(makelaars)
    logger.info(
        f"Processing {len(makelaars)} brokers with {max_workers} worker threads"
    )

    alle_nieuwe_properties, alle_verwijderde_properties = (
        communicatie.parallel_process_brokers(makelaars, max_workers=max_workers)
    )

    # After all scrapers have completed, synchronously apply database updates
    logger.info("All scrapers finished. Applying database updates synchronously...")
    communicatie.apply_database_updates(
        alle_nieuwe_properties, alle_verwijderde_properties
    )

    # Toon resultaten
    logger.info(f"Total new properties: {len(alle_nieuwe_properties)}")
    logger.info(f"Total removed properties: {len(alle_verwijderde_properties)}")

    # Verstuur mail met nieuwe woningen
    if alle_nieuwe_properties:
        logger.info(f"Sending email with {len(alle_nieuwe_properties)} new properties")

        # Initialiseer de mail service met de configuratie uit config.py
        mail_service = MailService()

        # Verstuur de mail met nieuwe woningen
        mail_success = mail_service.send_new_properties_email(
            None, alle_nieuwe_properties
        )

        if mail_success:
            logger.info(
                f"E-mail with {len(alle_nieuwe_properties)} new properties successfully sent to {len(EMAIL['recipients'])} recipient(s)"
            )
        else:
            logger.error("There was a problem sending the email.")

    # Log application end
    log_service.log_app_end(
        len(alle_nieuwe_properties), len(alle_verwijderde_properties)
    )


run_scraper_proces()

2025-05-13 18:10:12,851 - [MainThread] - root - INFO - ===== HuurhuisWebscraper Started =====
2025-05-13 18:10:12,852 - [MainThread] - root - INFO - Start time: 2025-05-13 18:10:12
2025-05-13 18:10:12,853 - [MainThread] - HuurhuisWebscraper - INFO - Starting multithreaded scraper process
2025-05-13 18:10:12,853 - [MainThread] - HuurhuisWebscraper - INFO - Processing 7 brokers with 7 worker threads
2025-05-13 18:10:12,854 - [MainThread] - connector - INFO - Starting parallel processing of 7 brokers with max_workers=7
2025-05-13 18:10:12,852 - [MainThread] - root - INFO - Start time: 2025-05-13 18:10:12
2025-05-13 18:10:12,853 - [MainThread] - HuurhuisWebscraper - INFO - Starting multithreaded scraper process
2025-05-13 18:10:12,853 - [MainThread] - HuurhuisWebscraper - INFO - Processing 7 brokers with 7 worker threads
2025-05-13 18:10:12,854 - [MainThread] - connector - INFO - Starting parallel processing of 7 brokers with max_workers=7
2025-05-13 18:10:12,856 - [Thread-Van Roomen Van d

2025-05-13 18:10:12,871 - [Thread-Zonnenberg Makelaardij] - connector - INFO - Verwerken van makelaar: Zonnenberg Makelaardij op thread Thread-Zonnenberg Makelaardij
2025-05-13 18:10:12,871 - [Thread-Ditters Makelaars] - connector - INFO - Verwerken van makelaar: Ditters Makelaars op thread Thread-Ditters Makelaars
2025-05-13 18:10:12,873 - [Thread-InterHouse Utrecht] - connector - INFO - Verwerken van makelaar: InterHouse Utrecht op thread Thread-InterHouse Utrecht
2025-05-13 18:10:12,871 - [Thread-Ditters Makelaars] - connector - INFO - Verwerken van makelaar: Ditters Makelaars op thread Thread-Ditters Makelaars
2025-05-13 18:10:12,873 - [Thread-InterHouse Utrecht] - connector - INFO - Verwerken van makelaar: InterHouse Utrecht op thread Thread-InterHouse Utrecht
2025-05-13 18:10:12,873 - [Thread-InterHouse Amersfoort] - connector - INFO - Verwerken van makelaar: InterHouse Amersfoort op thread Thread-InterHouse Amersfoort
2025-05-13 18:10:12,876 - [Thread-VastgoedNederland Veenendaa